In [1]:
!nvidia-smi

Sat Jul 24 04:59:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
!pip install -qq tokenizers===0.10.3

In [4]:
!pip install -qq transformers

In [5]:
!pip install -qq simpletransformers

In [6]:
from transformers import AutoTokenizer, AutoModel

In [7]:
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [8]:
pretrained = "monsoon-nlp/bert-base-thai"

In [9]:
model_args = {
    'manual_seed' : 0,
    'reprocess_input_data': False,
    
    'train_batch_size': 16,
    'eval_batch_size' : 8,
    
    'use_early_stopping': True,
    'early_stopping_delta': 0.01,
    'early_stopping_metric': 'eval_loss',
    'early_stopping_metric_minimize': True,
    'early_stopping_patience': 5,
    
    'evaluate_during_training' : True,
    'evaluate_during_training_verbose': True,
    
    'fp16': True,
    
    'num_train_epochs': 10,
    
    'overwrite_output_dir': True,

    'save_model_every_epoch': False,
    'save_steps':-1,
    
    'use_cached_eval_features' : True, 
    
    'max_seq_length': 256,
    'no_cache': False,

    'custom_parameter_groups':[{
        'params': ['classifier.weight', 'bert.encoder.layer.10.output.dense.weight'],
        'lr': 5e-5,
    }]
}

In [11]:
bert = QuestionAnsweringModel(
    'bert', 
    pretrained,
    use_cuda=torch.cuda.is_available(), 
    args=model_args
)

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at monsoon-nlp/bert-base-thai were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from th

Downloading:   0%|          | 0.00/489k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/113 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [13]:
import json
with open('/kaggle/input/moddataset/scg/train.json', 'r') as file:
  scgtrain = json.load(file, encoding='utf-8')
with open('/kaggle/input/moddataset/scg/eval.json', 'r') as file:
  scgeval = json.load(file, encoding='utf-8')
with open('/kaggle/input/moddataset/wiki/train.json','r') as file:
  wikitrain = json.load(file, encoding='utf-8')
with open('/kaggle/input/moddataset/wiki/eval.json','r') as file:
  wikieval = json.load(file, encoding='utf-8')
train = scgtrain + wikitrain
eval = scgeval + wikieval

In [14]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

In [ ]:
train = train[0:1000]

In [ ]:
eval = eval[0:200]

In [15]:
step, train_result = bert.train_model(train, eval_data=eval, show_running_loss=True)


convert squad examples to features:   0%|          | 0/12247 [00:00<?, ?it/s]Could not find answer: '2 ล้านพิกเซล, บันทึกวิดีโอและสตรีมมิ่ง, เครื่องเล่นเพลงดิจิตอลและวิทยุ FM' vs. '2 ล้านพิกเซล'

convert squad examples to features:   0%|          | 1/12247 [03:22<689:18:07, 202.64s/it]Could not find answer: 'จนกระทั่งเมื่อวันที่ 19 มกราคม พ.ศ. 2539 กระทรวงมหาดไทยได้ประกาศเปลี่ยนแปลงฐานะเป็น องค์การบริหารส่วนตำบลโจดหนองแก' vs. '19 มกราคม พ.ศ. 2539'

convert squad examples to features: 100%|██████████| 12247/12247 [05:57<00:00, 34.28it/s] 

add example index and unique id: 100%|██████████| 12247/12247 [00:00<00:00, 525785.25it/s]


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/1066 [00:00<?, ?it/s]



convert squad examples to features:   0%|          | 0/3042 [00:00<?, ?it/s]

convert squad examples to features:   0%|          | 1/3042 [00:44<37:42:23, 44.64s/it]

convert squad examples to features: 100%|██████████| 3042/3042 [01:26<00:00, 35.21it/s]


add example index and unique id: 100%|██████████| 3042/3042 [00:00<00:00, 442931.08it/s]


Running Evaluation:   0%|          | 0/520 [00:00<?, ?it/s]

...

Running Epoch 8 of 10:   0%|          | 0/1066 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/520 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/1066 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/520 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/520 [00:00<?, ?it/s]

In [16]:
eval_result, eval_texts = bert.eval_model(eval)

Running Evaluation:   0%|          | 0/520 [00:00<?, ?it/s]

In [17]:
eval_result

{'correct': 0, 'similar': 2, 'incorrect': 5, 'eval_loss': -5.270695612980769}

In [ ]:
eval_texts

In [18]:
with open('/kaggle/input/moddataset/test/test.json', 'r') as file:
  test = json.load(file, encoding='utf-8')

In [19]:
answers, probabilities = bert.predict([test[0]])


convert squad examples to features: 100%|██████████| 1/1 [00:00<00:00, 110.16it/s]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 9383.23it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
output = dict()
for i in test:
  answers,probabilities = bert.predict([i])
  for j in range(len(answers)):
    print(i['qas'][j]['question_id'])
    question_id = i['qas'][j]['question_id']
    answer = answers[j]['answer'][0]
    output[question_id] = [question_id,answer]


convert squad examples to features: 100%|██████████| 1/1 [00:00<00:00, 226.88it/s]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 9845.78it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

5507



convert squad examples to features: 100%|██████████| 1/1 [00:00<00:00, 141.17it/s]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 8648.05it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

5161



convert squad examples to features: 100%|██████████| 1/1 [00:00<00:00, 29.04it/s]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 8224.13it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

5162


In [21]:
output

{'5507': ['5507', '“WEDO Young Talent Innoprise”'],
 '5523': ['5523', '#DigitalOffice #GoBigRed #DataScience'],
 '5534': ['5534', 'เก้าอี้ ก็มีให้ครบ 4)'],
 '5561': ['5561', ''],
 '5567': ['5567', 'ขาว'],
 '5583': ['5583', ''],
 '5589': ['5589', ''],
 '5592': ['5592',
  'ขาว หรือ ดำ เกินไป Binary เกินไป มันอาจจะเหมาะในยุคที่ผ่านมา'],
 '5596': ['5596',
  '\ufeffเจาะกระบวนการใช้ OKR ตั้งโจทย์ที่ถูกต้อง ในเวลาที่ถูกต้อง กับ ‘อาร์ท'],
 '5602': ['5602', ''],
 '485': ['485', 'ราชวงศ์ตองอู'],
 '486': ['486', 'ราชวงศ์ตองอู'],
 '487': ['487', '2'],
 '488': ['488', '2'],
 '489': ['489', 'เพ็ชร'],
 '490': ['490',
  'ออกฉายในปี ค.ศ. 1997 ผลิตโดย ทเวนตีส์เซ็นจูรีฟ็อกซ์ และ พาราเมาต์พิกเจอส์ นำแสดงโดย ลีโอนาร์โด ดีคาปริโอ และ เคต'],
 '491': ['491',
  'ทัชมาฮาล ทัชมาฮาล (ภาษาฮินดี : ताजमहल, ตาช มฮัล รากศัพท์เดิมมาจากภาษาอาหรับ ตาจญ์ (มงกุฎ) และ มะฮัล (สถาน) (ภาษาอาหรับ : تاج محل ) เป็นอนุสรณ์สถาน ตั้งอยู่ในเมืองอัครา ประเทศอินเดีย นับเป็นหนึ่งในเจ็ดสิ่งมหัศจรรย์ของโลกยุคใหม่ประวัติ ประวัติ. ทัชมาฮาล 

In [22]:
df = []
for i in range(1,5609):
  if str(i) in output:
    df.append([i,output[str(i)][1]])
  else:
    df.append([i,''])

In [24]:
import pandas as pd
import numpy as np
outdf = pd.DataFrame(np.array(df),columns=['id','answer'])


In [29]:
outdf.to_csv('/kaggle/working/output.csv',index=False,header=True)